<a href="https://colab.research.google.com/github/YoonHoJeong/transformer-practice/blob/master/Transformer_Encoder_clone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install sentencepiece

In [15]:
import os
import numpy as np
import math
import matplotlib.pyplot as plt
import sentencepiece as spm

import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
vocab_file = "/content/drive/My Drive/Transformer-Encoder/models/kowiki.model"

In [8]:
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

True

In [9]:
lines = [
  "겨울은 추워요.",
  "감기 조심하세요."
]

In [12]:
# text를 tensor로 변환
inputs = []
for line in lines:
  pieces = vocab.encode_as_pieces(line)
  ids = vocab.encode_as_ids(line)
  inputs.append(torch.tensor(ids))
  print(pieces)

['▁겨울', '은', '▁추', '워', '요', '.']
['▁감', '기', '▁조', '심', '하', '세', '요', '.']


In [13]:
# 입력 길이가 다르므로 입력 최대 길이에 맟춰 padding(0)을 추가 해 줌
inputs = torch.nn.utils.rnn.pad_sequence(inputs, batch_first=True, padding_value=0)
# shape
print(inputs.size())
# 값
print(inputs)

torch.Size([2, 8])
tensor([[3091, 3604,  206, 3958, 3760, 3590,    0,    0],
        [ 212, 3605,   53, 3832, 3596, 3682, 3760, 3590]])


# Input Embedding

In [16]:
n_vocab = len(vocab) # vocab count
d_hidn = 128 # hidden size
nn_emb = nn.Embedding(n_vocab, d_hidn) # embedding 객체

input_embs = nn_emb(inputs) # input embedding
print(input_embs.size())

torch.Size([2, 8, 128])


- nn.Embedding(input-len, d-hidden)
  - 위 함수가 어떤 역할을 수행하는지 알아볼 것
  - 아마, d_hidden에 맞춰 input을 embedding으로 만들어주는 미리 학습된 모델이 아닐까 싶음

https://wikidocs.net/64779
- 파이토치에서는 임베딩 벡터를 사용하는 방법이 크게 2가지가 있다.
  1. embedding layer를 구성, 훈련 데이터로부터 처음부터 임베딩 벡터를 학습하는 방법
    - **nn.Embedding()**
  2. 훈련된 임베딩 벡터들을 가져와 사용하는 방법.

- 임베딩 층의 입력 : 입력 시퀀스의 단어들은 모두 정수 인코딩이 되어있어야 한다.
  - 특정 단어 → 단어에 부여된 고유 정수값 → 임베딩 층 통과 → 밀집 벡터

- 임베딩 층은 input 정수에 대해 밀집 벡터(dense vector)로 맵핑, 이 밀집 벡터는 기존 신경망에서 학습되듯 훈련된다.
- 특정 단어는 특정 임베딩 벡터와 맵핑되므로, 갖고 있는 단어에 대한 lookup table이 존재하게 된다.
- 해당 단어와 연결된 **임베딩 벡터는 임베딩 층에서 모델의 입력이 되어 역전파 과정에 의해 학습된다.**
- 파이토치는 단어 → 정수 인덱스 이후 **one-hot encoding을 수행하지 않는다.** 

- nn.Embedding(num_embeddings, embedding_dim, padding_idx)
  - nn.Embedding은 크게 2가지 인자를 받는다.
  1. num_embeddings
    : 임베딩할 단어들의 개수.
  2. embedding_dim
    : 임베딩할 벡터의 차원
  

# Positional Embedding
- 논문에서, "임베딩 벡터 내의 차원 인덱스"가 홀수, 짝수인 


In [19]:
""" sinusoid position embedding """
def get_sinusoid_encoding_table(n_seq, d_hidn):
    def cal_angle(position, i_hidn):
        return position / np.power(10000, 2 * (i_hidn // 2) / d_hidn)
    def get_posi_angle_vec(position):
        return [cal_angle(position, i_hidn) for i_hidn in range(d_hidn)]

    sinusoid_table = np.array([get_posi_angle_vec(i_seq) for i_seq in range(n_seq)])
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # dim 2i
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # dim 2i+1

    return sinusoid_table

- positional embedding
  1. sequence의 길이
  2. d_hidden : 은닉벡터의 차원
  - 이 외의 **임베딩이나, 단어의 정보는 사용하지 않는다**



In [20]:
n_seq = 64
pos_encoding = get_sinusoid_encoding_table(n_seq, d_hidn)

print (pos_encoding.shape) # 크기 출력

(64, 128)


In [22]:
pos_encoding = torch.FloatTensor(pos_encoding)

# positional encoding 값을 embedding instance로 만들어준다.
nn_pos = nn.Embedding.from_pretrained(pos_encoding, freeze=True)

positions = torch.arange(inputs.size(1), device=inputs.device, dtype=inputs.dtype).expand(inputs.size(0), inputs.size(1)).contiguous() + 1
pos_mask = inputs.eq(0) # input 값 중 0으로 되어있는 padding을 찾는다.

positions.masked_fill_(pos_mask, 0) # 
pos_embs = nn_pos(positions) # position embedding

from_pretrained() 함수를 통해, pos_encoding 값을 통해 nn_pos라는 layer embedding layer를 구축.

freeze = True로 설정했을 때, 학습되지 않는다. 그냥 embedding instance를 생성해준다.

[torch.arange()](https://pytorch.org/docs/stable/generated/torch.arange.html) : 주어진 범위 내의 정수를 순서대로 생성.



In [23]:
""" positional embedding과 input embedding을 말그대로 더해준다.
 두개의 임베딩은 같은 차원이기 때문에 가능 """
input_sums = input_embs + pos_embs


# 3. Scaled Dot Product Attention

In [24]:
# self-attention이므로 Q,K,V 모두가 똑같음.
Q = input_sums
K = input_sums
V = input_sums

# padding으로 설정했던 부분은 masked.
attn_mask = inputs.eq(0).unsqueeze(1).expand(Q.size(0), Q.size(1), K.size(1))



In [25]:
# matrix multiplication, Q * K^T
scores = torch.matmul(Q, K.transpose(-1, -2))
print(scores.size())
print(scores[0])

torch.Size([2, 8, 8])
tensor([[220.7007,  62.3027,  52.2023,  43.3869,  37.7986,  70.7991,  63.8554,
          63.8554],
        [ 62.3027, 140.0178,  42.8987,  36.1452,  28.2566,  43.7788,  57.3529,
          57.3529],
        [ 52.2023,  42.8987, 197.8656,  36.3456,  71.4895,  76.1569,  57.1159,
          57.1159],
        [ 43.3869,  36.1452,  36.3456, 152.3838,  69.6505,  43.8249,  32.9101,
          32.9101],
        [ 37.7986,  28.2566,  71.4895,  69.6505, 164.1191,  58.0175,  46.9227,
          46.9227],
        [ 70.7991,  43.7788,  76.1569,  43.8249,  58.0175, 223.8021,  46.2688,
          46.2688],
        [ 63.8554,  57.3529,  57.1159,  32.9101,  46.9227,  46.2688, 223.7537,
         223.7537],
        [ 63.8554,  57.3529,  57.1159,  32.9101,  46.9227,  46.2688, 223.7537,
         223.7537]], grad_fn=<SelectBackward>)


In [26]:
# scaling
d_head = 64
scores = scores.mul_(1/d_head**0.5)
print(scores.size())
print(scores[0])

torch.Size([2, 8, 8])
tensor([[27.5876,  7.7878,  6.5253,  5.4234,  4.7248,  8.8499,  7.9819,  7.9819],
        [ 7.7878, 17.5022,  5.3623,  4.5181,  3.5321,  5.4723,  7.1691,  7.1691],
        [ 6.5253,  5.3623, 24.7332,  4.5432,  8.9362,  9.5196,  7.1395,  7.1395],
        [ 5.4234,  4.5181,  4.5432, 19.0480,  8.7063,  5.4781,  4.1138,  4.1138],
        [ 4.7248,  3.5321,  8.9362,  8.7063, 20.5149,  7.2522,  5.8653,  5.8653],
        [ 8.8499,  5.4723,  9.5196,  5.4781,  7.2522, 27.9753,  5.7836,  5.7836],
        [ 7.9819,  7.1691,  7.1395,  4.1138,  5.8653,  5.7836, 27.9692, 27.9692],
        [ 7.9819,  7.1691,  7.1395,  4.1138,  5.8653,  5.7836, 27.9692, 27.9692]],
       grad_fn=<SelectBackward>)


In [28]:
scores.masked_fill_(attn_mask, -1e9)
print(scores.size())
print(scores[0])

torch.Size([2, 8, 8])
tensor([[ 2.7588e+01,  7.7878e+00,  6.5253e+00,  5.4234e+00,  4.7248e+00,
          8.8499e+00, -1.0000e+09, -1.0000e+09],
        [ 7.7878e+00,  1.7502e+01,  5.3623e+00,  4.5181e+00,  3.5321e+00,
          5.4723e+00, -1.0000e+09, -1.0000e+09],
        [ 6.5253e+00,  5.3623e+00,  2.4733e+01,  4.5432e+00,  8.9362e+00,
          9.5196e+00, -1.0000e+09, -1.0000e+09],
        [ 5.4234e+00,  4.5181e+00,  4.5432e+00,  1.9048e+01,  8.7063e+00,
          5.4781e+00, -1.0000e+09, -1.0000e+09],
        [ 4.7248e+00,  3.5321e+00,  8.9362e+00,  8.7063e+00,  2.0515e+01,
          7.2522e+00, -1.0000e+09, -1.0000e+09],
        [ 8.8499e+00,  5.4723e+00,  9.5196e+00,  5.4781e+00,  7.2522e+00,
          2.7975e+01, -1.0000e+09, -1.0000e+09],
        [ 7.9819e+00,  7.1691e+00,  7.1395e+00,  4.1138e+00,  5.8653e+00,
          5.7836e+00, -1.0000e+09, -1.0000e+09],
        [ 7.9819e+00,  7.1691e+00,  7.1395e+00,  4.1138e+00,  5.8653e+00,
          5.7836e+00, -1.0000e+09, -1.0000e

In [29]:
# softmax, weight distribution을 구한다.
attn_prob = nn.Softmax(dim=-1)(scores)
print(attn_prob.size())
print(attn_prob[0])

torch.Size([2, 8, 8])
tensor([[1.0000e+00, 2.5181e-09, 7.1246e-10, 2.3670e-10, 1.1771e-10, 7.2832e-09,
         0.0000e+00, 0.0000e+00],
        [6.0404e-05, 9.9993e-01, 5.3418e-06, 2.2964e-06, 8.5666e-07, 5.9629e-06,
         0.0000e+00, 0.0000e+00],
        [1.2371e-08, 3.8667e-09, 1.0000e+00, 1.7045e-09, 1.3786e-07, 2.4707e-07,
         0.0000e+00, 0.0000e+00],
        [1.2103e-06, 4.8951e-07, 5.0193e-07, 9.9996e-01, 3.2260e-05, 1.2784e-06,
         0.0000e+00, 0.0000e+00],
        [1.3882e-07, 4.2116e-08, 9.3633e-06, 7.4404e-06, 9.9998e-01, 1.7381e-06,
         0.0000e+00, 0.0000e+00],
        [4.9426e-09, 1.6870e-10, 9.6563e-09, 1.6967e-10, 1.0002e-09, 1.0000e+00,
         0.0000e+00, 0.0000e+00],
        [4.7024e-01, 2.0860e-01, 2.0251e-01, 9.8264e-03, 5.6636e-02, 5.2191e-02,
         0.0000e+00, 0.0000e+00],
        [4.7024e-01, 2.0860e-01, 2.0251e-01, 9.8264e-03, 5.6636e-02, 5.2191e-02,
         0.0000e+00, 0.0000e+00]], grad_fn=<SelectBackward>)


In [30]:
# weight distribution, value를 통해 가중 합을 구한다.
context = torch.matmul(attn_prob, V)
print(context.size())

torch.Size([2, 8, 128])


## ScaledDotProductAttention
- 위의 과정을 하나의 class로 만들어 사용한다.

In [37]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, d_head):
        super().__init__()
        self.scale = 1 / (d_head ** 0.5)
    
    def forward(self, Q, K, V, attn_mask):
        # (bs, n_head, n_q_seq, n_k_seq)
        scores = torch.matmul(Q, K.transpose(-1, -2)).mul_(self.scale)
        scores.masked_fill_(attn_mask, -1e9)
        # (bs, n_head, n_q_seq, n_k_seq)
        attn_prob = nn.Softmax(dim=-1)(scores)
        # (bs, n_head, n_q_seq, d_v)
        context = torch.matmul(attn_prob, V)
        # (bs, n_head, n_q_seq, d_v), (bs, n_head, n_q_seq, n_v_seq)
        return context, attn_prob

# 4. Multi-Head Attention
- Q, K, V, attn_mask는 Scaled Dot Product Attention과 동일.
- 여기선 head의 개수는 2개로 head의 dimension은 128/2 = 64가 된다.

In [31]:
Q = input_sums
K = input_sums
V = input_sums
attn_mask = inputs.eq(0).unsqueeze(1).expand(Q.size(0), Q.size(1), K.size(1))

batch_size = Q.size(0)
n_head = 2

- Q, K, V를 Multi Head로 만드는 과정.

In [32]:
W_Q = nn.Linear(d_hidn, n_head * d_head)
W_K = nn.Linear(d_hidn, n_head * d_head)
W_V = nn.Linear(d_hidn, n_head * d_head)

https://stackoverflow.com/questions/54916135/what-is-the-class-definition-of-nn-linear-in-pytorch

- nn.Linear(in_features, out_features, bias=True)
  - 주어진 데이터에 대해 linear transformation을 적용한다.
  - in_features : input sample의 사이즈
  - out_features : output sample의 사이즈
  - bias
- (y = wx + b)에서, linear transformation의 weight, bias 는 랜덤하게 초기화된다.
  - fully connected layer, 입력값을 동일하게 넣어도 다른 결과값이 나오게 됨. Q,K,V 각각의 Weight Matrix를 다르게 만들기 위한 방법으로 활용.

- 위 코드
  - in_features : 128(hidden feature dimension)
  - out_features : 2 * 64 

In [34]:
q_s = W_Q(Q).view(batch_size, -1, n_head, d_head).transpose(1,2)
# (bs, n_head, n_seq, d_head)
k_s = W_K(K).view(batch_size, -1, n_head, d_head).transpose(1,2)
# (bs, n_head, n_seq, d_head)
v_s = W_V(V).view(batch_size, -1, n_head, d_head).transpose(1,2)
print(q_s.size(), k_s.size(), v_s.size())

torch.Size([2, 2, 8, 64]) torch.Size([2, 2, 8, 64]) torch.Size([2, 2, 8, 64])


- 기존의 128 크기를 갖는 q_s를 랜덤하게 선형 변환시킨 q_s가 나오고, 이를 view()함수를 통해 multi-head로 바꿔줌.

-  [tensor.view()](https://pytorch.org/docs/stable/tensor_view.html) : base tensor와 같은 데이터를 공유.
  - 

In [35]:
# attention mask도 똑같이 변경.
attn_mask = attn_mask.unsqueeze(1).repeat(1, n_head, 1, 1)

In [38]:
scaled_dot_attn = ScaledDotProductAttention(d_head)
context, attn_prob = scaled_dot_attn(q_s, k_s, v_s, attn_mask)

In [39]:
print(context.size())
print(attn_prob.size())

torch.Size([2, 2, 8, 64])
torch.Size([2, 2, 8, 8])


### concat

In [40]:
context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_head * d_head)

### Linear Transformation (W_0)

In [41]:
linear = nn.Linear(n_head * d_head, d_hidn)
# (bs, n_seq, d_hidn)
output = linear(context)
print(output.size())

torch.Size([2, 8, 128])


## MultiHeadAttention Class
- 위의 과정을 Class로 만들어 사용.

In [ ]:
""" multi head attention """
class MultiHeadAttention(nn.Module):
    def __init__(self, d_hidn, n_head, d_head):
        super().__init__()
        self.d_hidn = d_hidn
        self.n_head = n_head
        self.d_head = d_head

        self.W_Q = nn.Linear(d_hidn, n_head * d_head)
        self.W_K = nn.Linear(d_hidn, n_head * d_head)
        self.W_V = nn.Linear(d_hidn, n_head * d_head)
        self.scaled_dot_attn = ScaledDotProductAttention(d_head)
        self.linear = nn.Linear(n_head * d_head, d_hidn)
    
    def forward(self, Q, K, V, attn_mask):
        batch_size = Q.size(0)
        # (bs, n_head, n_q_seq, d_head)
        q_s = self.W_Q(Q).view(batch_size, -1, self.n_head, self.d_head).transpose(1,2)
        # (bs, n_head, n_k_seq, d_head)
        k_s = self.W_K(K).view(batch_size, -1, self.n_head, self.d_head).transpose(1,2)
        # (bs, n_head, n_v_seq, d_head)
        v_s = self.W_V(V).view(batch_size, -1, self.n_head, self.d_head).transpose(1,2)

        # (bs, n_head, n_q_seq, n_k_seq)
        attn_mask = attn_mask.unsqueeze(1).repeat(1, self.n_head, 1, 1)

        # (bs, n_head, n_q_seq, d_head), (bs, n_head, n_q_seq, n_k_seq)
        context, attn_prob = self.scaled_dot_attn(q_s, k_s, v_s, attn_mask)
        # (bs, n_head, n_q_seq, h_head * d_head)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.n_head * self.d_head)
        # (bs, n_head, n_q_seq, e_embd)
        output = self.linear(context)
        # (bs, n_q_seq, d_hidn), (bs, n_head, n_q_seq, n_k_seq)
        return output, attn_prob

# 5. Masked MultiHeadAttention (생략)
- encoder에서는 안 쓰여서 일단 생략함

# 6. FeedForward Neural Network(FFNN)
- W0, W1을 통해 구성한 Feed-Forward NN
- activation layer로 넣은듯.

In [44]:
conv1 = nn.Conv1d(in_channels=d_hidn, out_channels=d_hidn * 4, kernel_size=1)
# (bs, d_hidn * 4, n_seq)
ff_1 = conv1(output.transpose(1, 2))
print(ff_1.size())

torch.Size([2, 512, 8])


-

In [45]:
active = F.gelu
ff_2 = active(ff_1)

In [46]:
conv2 = nn.Conv1d(in_channels=d_hidn * 4, out_channels=d_hidn, kernel_size=1)
ff_3 = conv2(ff_2).transpose(1, 2)
print(ff_3.size())

torch.Size([2, 8, 128])


## Feed Forward NN Class

In [47]:
""" feed forward """
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self, d_hidn):
        super().__init__()

        self.conv1 = nn.Conv1d(in_channels=self.config.d_hidn, out_channels=self.config.d_hidn * 4, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=self.config.d_hidn * 4, out_channels=self.config.d_hidn, kernel_size=1)
        self.active = F.gelu

    def forward(self, inputs):
        # (bs, d_ff, n_seq)
        output = self.active(self.conv1(inputs.transpose(1, 2)))
        # (bs, n_seq, d_hidn)
        output = self.conv2(output).transpose(1, 2)
        # (bs, n_seq, d_hidn)
        return output